![GMV](https://www.gmv.com/export/system/modules/com.gmv.teresa.site/resources/theme/img/logo_gmv.svg)  ![Apache Spark](http://spark.apache.org/images/spark-logo.png)

# KDD99 Supervised Learning I

# Apache Spark Initialization

In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="SecurityDataScience")

In [2]:
%matplotlib inline

## 0. Libraries

In [3]:
import numpy as np
import pandas as pd
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import * 
sqlContext = SQLContext(sc)

## 1. Data Description

**Intrinsic attributes**

These attributes are extracted from the headers' area of the network packets.

Col|Feature name  | description |	type
---|--------------|-------------|------------
1  |duration 	  |length (number of seconds) of the connection |continuous
2  |protocol_type |type of the protocol, e.g. tcp, udp, etc. |discrete
3  |service 	  |network service on the destination, e.g., http, telnet, etc. |discrete
4  |flag 	      |normal or error status of the connection. The possible status are this: SF, S0, S1, S2, S3, OTH, REJ, RSTO, RSTOS0, SH, RSTRH, SHR 	|discrete 
5  |src_bytes 	  |number of data bytes from source to destination 	|continuous
6  |dst_bytes 	  |number of data bytes from destination to source 	|continuous
7  |land 	      |1 if connection is from/to the same host/port; 0 otherwise 	|discrete
8  |wrong_fragment|sum of bad checksum packets in a connection 	|continuous
9  |urgent 	      |number of urgent packets. Urgent packets are packets with the urgent bit activated 	|continuous


**Class attribute**

The 42nd attribute is the ***class_attack*** attribute, it indicates which type of connections is each instance: normal or which attack. The values it can take are the following: *anomaly, dict, dict_simple, eject, eject-fail, ffb, ffb_clear, format, format_clear, format-fail, ftp-write, guest, imap, land, load_clear, loadmodule, multihop, perl_clear, perlmagic, phf, rootkit, spy, syslog, teardrop, warez, warezclient, warezmaster, pod, back, ip- sweep, neptune, nmap, portsweep, satan, smurf and normal*.

** Categories of class attribute **


class_attack |Category
-------|--------------
smurf| dos
neptune| dos
back| dos
teardrop| dos
pod| dos
land| dos
normal|normal
satan|probe
ipsweep|probe
portsweep|probe
nmap|probe
warezclient|r2l
guess_passwd|r2l
warezmaster|r2l
imap|r2l
ftp_write|r2l
multihop|r2l
phf|r2l
spy|r2l
buffer_overflow|u2r
rootkit|u2r
loadmodule|u2r
perl|u2r

## 2. Load Data

In [4]:
textFileConn = sc.textFile('./data/KDD/KDDTrain+.txt', 4)


In [5]:
#Creating the schema

#we define the name of the columns

columnNames=["class_attack", "duration","protocol_type","service","flag","src_bytes","dst_bytes","land",
                 "wrong_fragment","urgent"]

In [6]:
#quick fields initialitation all for FloatType
connFields = [StructField(colName, FloatType(), True) for colName in columnNames]

In [7]:
#we proceed to modify the respective fields so that they reflect the correct data type:
connFields[0].dataType = StringType()
connFields[2].dataType = StringType()
connFields[3].dataType = StringType()
connFields[4].dataType = StringType()

In [8]:
# we can construct our schema, which we will use later below for building the data frame
connSchema = StructType(connFields)

In [9]:
#Parsing the file
def parseReg(p):
    return ( p[41]
            ,float(p[0])
            ,p[1], p[2], p[3] 
            ,float(p[4])
            ,float(p[5])
            ,float(p[6])
            ,float(p[7])
            ,float(p[8])
            )

In [10]:
connParsedFile = (textFileConn.map(lambda line: line.split(','))
                              .map(parseReg))

In [11]:
# We are now ready to build our data frame, using the connParsedFile RDD computed above and the schema 
# variable already calculated:
conn = sqlContext.createDataFrame(connParsedFile, connSchema)
conn.cache()

DataFrame[class_attack: string, duration: float, protocol_type: string, service: string, flag: string, src_bytes: float, dst_bytes: float, land: float, wrong_fragment: float, urgent: float]

In [12]:
conn.take(3)

[Row(class_attack='normal', duration=0.0, protocol_type='tcp', service='ftp_data', flag='SF', src_bytes=491.0, dst_bytes=0.0, land=0.0, wrong_fragment=0.0, urgent=0.0),
 Row(class_attack='normal', duration=0.0, protocol_type='udp', service='other', flag='SF', src_bytes=146.0, dst_bytes=0.0, land=0.0, wrong_fragment=0.0, urgent=0.0),
 Row(class_attack='neptune', duration=0.0, protocol_type='tcp', service='private', flag='S0', src_bytes=0.0, dst_bytes=0.0, land=0.0, wrong_fragment=0.0, urgent=0.0)]

In [13]:
conn.limit(4).toPandas()

,class_attack,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent
0,normal,0.0,tcp,ftp_data,SF,491.0,0.0,0.0,0.0,0.0
1,normal,0.0,udp,other,SF,146.0,0.0,0.0,0.0,0.0
2,neptune,0.0,tcp,private,S0,0.0,0.0,0.0,0.0,0.0
3,normal,0.0,tcp,http,SF,232.0,8153.0,0.0,0.0,0.0


In [14]:
#get all the distint values of class_attack
conn.select("class_attack").distinct().toPandas()

,class_attack
0,neptune
1,satan
2,nmap
3,portsweep
4,back
5,warezclient
6,guess_passwd
7,normal
8,rootkit
9,perl


-----------------

## 3. Data Preparation

### 3.1 Encoding categorical features

In [15]:
from pyspark.sql import functions as F

In [16]:
def encodeCategorical(df, catName):
    #Encode the categorical variable in different columns foreach categories 
    #and the value is equal to 1 if the category is equal to column name and 0 otherwise. 
    #Finally drops the categorical variable
    
    categories = df.select(catName).distinct().toPandas()[catName]
    aux = df
    for c in categories:
        aux = aux.withColumn(c, F.when(df[catName] == c, 1).otherwise(0))
        
    return aux.drop(catName)

### Encoding *protocol_type*

In [17]:
conn.select("protocol_type").distinct().toPandas()

,protocol_type
0,tcp
1,udp
2,icmp


In [18]:
connEncoded = encodeCategorical(conn, "protocol_type")

In [19]:
connEncoded.limit(10).toPandas()

,class_attack,duration,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,tcp,udp,icmp
0,normal,0.0,ftp_data,SF,491.0,0.0,0.0,0.0,0.0,1,0,0
1,normal,0.0,other,SF,146.0,0.0,0.0,0.0,0.0,0,1,0
2,neptune,0.0,private,S0,0.0,0.0,0.0,0.0,0.0,1,0,0
3,normal,0.0,http,SF,232.0,8153.0,0.0,0.0,0.0,1,0,0
4,normal,0.0,http,SF,199.0,420.0,0.0,0.0,0.0,1,0,0
5,neptune,0.0,private,REJ,0.0,0.0,0.0,0.0,0.0,1,0,0
6,neptune,0.0,private,S0,0.0,0.0,0.0,0.0,0.0,1,0,0
7,neptune,0.0,private,S0,0.0,0.0,0.0,0.0,0.0,1,0,0
8,neptune,0.0,remote_job,S0,0.0,0.0,0.0,0.0,0.0,1,0,0
9,neptune,0.0,private,S0,0.0,0.0,0.0,0.0,0.0,1,0,0


### Encoding *service*

In [20]:
connEncoded.select("service").distinct().toPandas()

,service
0,telnet
1,ftp
2,auth
3,iso_tsap
4,systat
5,name
6,sql_net
7,ntp_u
8,X11
9,pop_3


In [21]:
connEncoded = encodeCategorical(connEncoded, "service")

In [22]:
connEncoded.limit(10).toPandas()

,class_attack,duration,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,tcp,udp,...,http_443,echo,netstat,remote_job,gopher,imap4,uucp,ssh,rje,bgp
0,normal,0.0,SF,491.0,0.0,0.0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,normal,0.0,SF,146.0,0.0,0.0,0.0,0.0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,neptune,0.0,S0,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,normal,0.0,SF,232.0,8153.0,0.0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,normal,0.0,SF,199.0,420.0,0.0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,neptune,0.0,REJ,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,neptune,0.0,S0,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,neptune,0.0,S0,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,neptune,0.0,S0,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,0,1,0,0,0,0,0,0
9,neptune,0.0,S0,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,0,0,0,0,0,0,0,0


### Encoding *flag*

In [23]:
connEncoded.select("flag").distinct().toPandas()

,flag
0,RSTOS0
1,S3
2,SF
3,S0
4,OTH
5,REJ
6,RSTO
7,RSTR
8,SH
9,S2


In [24]:
connEncoded = encodeCategorical(connEncoded, "flag")

In [25]:
connEncoded.limit(10).toPandas()

,class_attack,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,tcp,udp,icmp,...,S3,SF,S0,OTH,REJ,RSTO,RSTR,SH,S2,S1
0,normal,0.0,491.0,0.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,normal,0.0,146.0,0.0,0.0,0.0,0.0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,neptune,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
3,normal,0.0,232.0,8153.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,normal,0.0,199.0,420.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
5,neptune,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
6,neptune,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
7,neptune,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
8,neptune,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
9,neptune,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


###  Encoding *class_attack* (**label**) like Integers

In [26]:
connEncoded.select("class_attack").distinct().toPandas()

,class_attack
0,neptune
1,satan
2,nmap
3,portsweep
4,back
5,warezclient
6,guess_passwd
7,normal
8,rootkit
9,perl


In [27]:
categories = connEncoded.select("class_attack").distinct().toPandas()["class_attack"]

In [28]:
dictCategories = dict((v,int(k)) for (k,v) in categories.to_dict().items())

In [29]:
from pyspark.sql.functions import udf

In [30]:
def categoriesToInt(cat):
    return dictCategories[cat]

udfCategoriesToInt = udf(categoriesToInt, IntegerType())

In [31]:
connEncoded = connEncoded.withColumn("class_attack", udfCategoriesToInt("class_attack") )

In [32]:
connEncoded.limit(10).toPandas()

,class_attack,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,tcp,udp,icmp,...,S3,SF,S0,OTH,REJ,RSTO,RSTR,SH,S2,S1
0,7,0.0,491.0,0.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,7,0.0,146.0,0.0,0.0,0.0,0.0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
3,7,0.0,232.0,8153.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,7,0.0,199.0,420.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
5,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
6,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
7,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
8,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
9,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


## 4. Decision Trees

### 4.1 Data preparation for Decision Trees

In [33]:
#Take 30% of the data to improve the performance
connSample = connEncoded.sample(withReplacement = True, fraction = 0.30)

In [34]:
#Modify class_attack catagory to binary category 
connDT = connSample.withColumn('class_attack', when(connEncoded['class_attack'] == 4, 0.0).otherwise(1.0))
connDT.cache()

DataFrame[class_attack: double, duration: float, src_bytes: float, dst_bytes: float, land: float, wrong_fragment: float, urgent: float, tcp: int, udp: int, icmp: int, telnet: int, ftp: int, auth: int, iso_tsap: int, systat: int, name: int, sql_net: int, ntp_u: int, X11: int, pop_3: int, ldap: int, discard: int, tftp_u: int, Z39_50: int, daytime: int, domain_u: int, login: int, smtp: int, http_2784: int, mtp: int, domain: int, http: int, harvest: int, link: int, courier: int, kshell: int, pop_2: int, other: int, exec: int, efs: int, nnsp: int, IRC: int, pm_dump: int, private: int, urh_i: int, ftp_data: int, whois: int, nntp: int, netbios_ns: int, aol: int, klogin: int, shell: int, red_i: int, tim_i: int, uucp_path: int, eco_i: int, vmnet: int, ctf: int, supdup: int, http_8001: int, finger: int, netbios_dgm: int, printer: int, urp_i: int, ecr_i: int, time: int, netbios_ssn: int, csnet_ns: int, hostnames: int, sunrpc: int, http_443: int, echo: int, netstat: int, remote_job: int, gopher: i

In [35]:
connDT.limit(10).toPandas()

,class_attack,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,tcp,udp,icmp,...,S3,SF,S0,OTH,REJ,RSTO,RSTR,SH,S2,S1
0,1.0,0.0,491.0,0.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1.0,0.0,287.0,2251.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,1.0,0.0,334.0,0.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1.0,0.0,233.0,616.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,1.0,0.0,233.0,616.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
5,1.0,0.0,343.0,1178.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
8,1.0,0.0,303.0,555.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
9,1.0,0.0,303.0,555.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [36]:
#Adapt de DataFrame to the algorithm API
from pyspark.ml.linalg import DenseVector

#Define a DataFrame with columns "label" and "features"
connDT = connDT.rdd.map(lambda row: (row[0], DenseVector(row[1:]))).toDF(['label','features'])

In [37]:
connDT.limit(10).toPandas()

,label,features
0,1.0,"[0.0, 491.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0..."
1,1.0,"[0.0, 287.0, 2251.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
2,1.0,"[0.0, 334.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0..."
3,1.0,"[0.0, 233.0, 616.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0..."
4,1.0,"[0.0, 233.0, 616.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0..."
5,1.0,"[0.0, 343.0, 1178.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
6,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
7,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
8,1.0,"[0.0, 303.0, 555.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0..."
9,1.0,"[0.0, 303.0, 555.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0..."


### 4.2 Training Decision Tree

**Parameters**:
* **Problem specification parameters** 
    * *algo*: Classification or Regression
    * *numClasses*: Number of classes (for Classification only)
    * *categoricalFeaturesInfo*: Specifies which features are categorical and how many categorical values each of those features can take. This is given as a map from feature indices to feature arity (number of categories). Any features not in this map are treated as continuous.
* **Stopping criteria**
    * *maxDepth*: Maximum depth of a tree. Deeper trees are more expressive (potentially allowing higher accuracy), but they are also more costly to train and are more likely to overfit.
    * *minInstancesPerNode*: For a node to be split further, each of its children must receive at least this number of training instances. This is commonly used with RandomForest since those are often trained deeper than individual trees.
    * *minInfoGain*: For a node to be split further, the split must improve at least this much (in terms of information gain).
* **Tunable parameters**
    * *maxBins*: Number of bins used when discretizing continuous features.
    * *maxMemoryInMB*: Amount of memory to be used for collecting sufficient statistics.
    * *impurity*: Impurity measure (Gini impurity, Entropy or Variance) used to choose between candidate splits. This measure must match the algo parameter.

In [38]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.evaluation import MulticlassMetrics

In [39]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(connDT)

In [40]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 20 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol='features', outputCol="indexedFeatures", maxCategories=20).fit(connDT)

In [41]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = connDT.randomSplit([0.7, 0.3])

In [42]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

In [43]:
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

In [44]:
# Train model with trainingData.  This also runs the indexers.
dtModel = pipeline.fit(trainingData)

In [45]:
# Make predictions with testData.
predictions = dtModel.transform(testData)

In [46]:
# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").limit(10).toPandas()

,prediction,indexedLabel,features
0,1.0,1.0,"[0.0, 50248.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
1,1.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
2,1.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
3,1.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
4,1.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
5,1.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
6,1.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
7,1.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
8,1.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
9,1.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."


In [47]:
# Compute raw scores on the test set
predictionAndLabels = predictions.select("prediction", "indexedLabel").rdd

In [48]:
predictionAndLabels.take(3)

[Row(prediction=1.0, indexedLabel=1.0),
 Row(prediction=1.0, indexedLabel=1.0),
 Row(prediction=1.0, indexedLabel=1.0)]

In [49]:
# Instantiate Basic Metrics object
basicMetrics = MulticlassMetrics(predictionAndLabels)

In [50]:
print("Summary Stats")
print("Weighted Precision = %s" % basicMetrics.weightedPrecision)
print("Weighted Recall = %s" % basicMetrics.weightedRecall)
print("Weighted F1 Score = %s" % basicMetrics.weightedFMeasure())
print("Confusion Matrix:")
pd.DataFrame(basicMetrics.confusionMatrix().toArray())

Summary Stats
Weighted Precision = 0.9999131913465277
Weighted Recall = 0.9999122268059334
Weighted F1 Score = 0.9999124673318651
Confusion Matrix:


,0,1
0,11302.0,1.0
1,0.0,90.0


In [51]:
# Instantiate Advanced Metrics object
advMetrics = BinaryClassificationMetrics(predictionAndLabels)

In [52]:
# Area under precision-recall curve
print("Area under PR = %s" % advMetrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % advMetrics.areaUnderROC)

Area under PR = 0.989010989010989
Area under ROC = 0.9999557639564717


In [53]:
# Show rules
treeModel = dtModel.stages[2]
print(treeModel.toDebugString)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_48d7b9cde909b87d02d4) of depth 4 with 9 nodes
  If (feature 1 <= 3584.0)
   Predict: 0.0
  Else (feature 1 > 3584.0)
   If (feature 30 in {0.0})
    Predict: 0.0
   Else (feature 30 not in {0.0})
    If (feature 2 <= 6313.0)
     If (feature 2 <= 3432.5)
      Predict: 1.0
     Else (feature 2 > 3432.5)
      Predict: 0.0
    Else (feature 2 > 6313.0)
     Predict: 1.0



### 4.3 Model Selection via Cross-Validation to Decision Tree

In [54]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [55]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(connDT)

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 20 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol='features', outputCol="indexedFeatures", maxCategories=20).fit(connDT)
    
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

In [56]:
grid = ParamGridBuilder().addGrid(dt.maxDepth, range(3,7)) \
                         .addGrid(dt.maxBins, range(20,51, 10)) \
                         .build()

In [57]:
evaluator = BinaryClassificationEvaluator(metricName="areaUnderPR")

In [58]:
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=grid, evaluator=evaluator)

In [59]:
%%time 
cvModel = cv.fit(connDT)

CPU times: user 2.14 s, sys: 1.05 s, total: 3.19 s
Wall time: 1min 20s


In [60]:
%%time
evaluator.evaluate(cvModel.transform(connDT))

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 1.8 s


0.5007880907498942

In [61]:
cvModel.bestModel.stages[2]

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_466885ac52cb301031d8) of depth 3 with 7 nodes

In [62]:
evaluator.evaluate(cvModel.transform(connDT), {evaluator.metricName: "areaUnderPR"})

0.5007880907498942

## 5. Random Forests
Random forests are ensembles of decision trees. Random forests are one of the most successful machine learning models for classification and regression. They combine many decision trees in order to **reduce the risk of overfitting**. Like decision trees, random forests handle categorical features, extend to the multiclass classification setting, do not require feature scaling, and are able to capture non-linearities and feature interactions.

Random forests train a set of decision trees separately, so the training can be done in parallel. The algorithm injects randomness into the training process so that each decision tree is a bit different. Combining the predictions from each tree reduces the variance of the predictions, improving the performance on test data

**Parameters:**
* **numTrees**: Number of trees in the forest. Increasing the number of trees will decrease the variance in predictions, improving the model’s test-time accuracy. Training time increases roughly linearly in the number of trees.
* **maxDepth**: Maximum depth of each tree in the forest. Increasing the depth makes the model more expressive and powerful. However, deep trees take longer to train and are also more prone to overfitting. In general, it is acceptable to train deeper trees when using random forests than when using a single decision tree. One tree is more likely to overfit than a random forest (because of the variance reduction from averaging multiple trees in the forest).
* **subsamplingRate**: This parameter specifies the size of the dataset used for training each tree in the forest, as a fraction of the size of the original dataset. The default (1.0) is recommended, but decreasing this fraction can speed up training.
* **featureSubsetStrategy**: Number of features to use as candidates for splitting at each tree node. The number is specified as a fraction or function of the total number of features. Decreasing this number will speed up training, but can sometimes impact performance if too low.

### 5.1 Data preparation for RandomForest

In [63]:
#Take 30% of the data to improve the performance
connSample = connEncoded.sample(withReplacement = True, fraction = 0.30)

In [64]:
#Modify class_attack catagory to binary category 
connRF = connSample.withColumn('class_attack', when(connEncoded['class_attack'] == 4, 0.0).otherwise(1.0))
connRF.cache()

DataFrame[class_attack: double, duration: float, src_bytes: float, dst_bytes: float, land: float, wrong_fragment: float, urgent: float, tcp: int, udp: int, icmp: int, telnet: int, ftp: int, auth: int, iso_tsap: int, systat: int, name: int, sql_net: int, ntp_u: int, X11: int, pop_3: int, ldap: int, discard: int, tftp_u: int, Z39_50: int, daytime: int, domain_u: int, login: int, smtp: int, http_2784: int, mtp: int, domain: int, http: int, harvest: int, link: int, courier: int, kshell: int, pop_2: int, other: int, exec: int, efs: int, nnsp: int, IRC: int, pm_dump: int, private: int, urh_i: int, ftp_data: int, whois: int, nntp: int, netbios_ns: int, aol: int, klogin: int, shell: int, red_i: int, tim_i: int, uucp_path: int, eco_i: int, vmnet: int, ctf: int, supdup: int, http_8001: int, finger: int, netbios_dgm: int, printer: int, urp_i: int, ecr_i: int, time: int, netbios_ssn: int, csnet_ns: int, hostnames: int, sunrpc: int, http_443: int, echo: int, netstat: int, remote_job: int, gopher: i

In [65]:
connRF.limit(10).toPandas()

,class_attack,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,tcp,udp,icmp,...,S3,SF,S0,OTH,REJ,RSTO,RSTR,SH,S2,S1
0,1.0,0.0,491.0,0.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1.0,0.0,287.0,2251.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
5,1.0,0.0,253.0,11905.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
7,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
8,1.0,0.0,241.0,1400.0,0.0,0.0,0.0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
9,1.0,0.0,45.0,45.0,0.0,0.0,0.0,0,1,0,...,0,1,0,0,0,0,0,0,0,0


In [66]:
#Adapt de DataFrame to the algorithm API
from pyspark.ml.linalg import DenseVector

#Define a DataFrame with columns "label" and "features"
connRF = connRF.rdd.map(lambda row: (row[0], DenseVector(row[1:]))).toDF(['label','features'])

In [67]:
connRF.limit(10).toPandas()

,label,features
0,1.0,"[0.0, 491.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0..."
1,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
2,1.0,"[0.0, 287.0, 2251.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
3,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
4,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
5,1.0,"[0.0, 253.0, 11905.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
6,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
7,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
8,1.0,"[0.0, 241.0, 1400.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
9,1.0,"[0.0, 45.0, 45.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0..."


### 5.2 RandomForest Model

In [68]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [69]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(connRF)

In [70]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 20 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol='features', outputCol="indexedFeatures", maxCategories=20).fit(connRF)

In [71]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = connRF.randomSplit([0.7, 0.3])

In [72]:
# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",
                           numTrees = 10, maxDepth = 5, impurity = "gini")

In [73]:
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

In [74]:
# Train model with trainingData.  This also runs the indexers.
rfModel = pipeline.fit(trainingData)

In [75]:
# Make predictions with testData.
predictions = rfModel.transform(testData)

In [76]:
# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").limit(10).toPandas()

,prediction,indexedLabel,features
0,0.0,1.0,"[0.0, 53452.0, 7300.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
1,0.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
2,0.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
3,0.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
4,0.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
5,0.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
6,0.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
7,0.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
8,0.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."
9,0.0,1.0,"[0.0, 54540.0, 8314.0, 0.0, 0.0, 0.0, 1.0, 0.0..."


In [77]:
# Compute raw scores on the test set
predictionAndLabels = predictions.select("prediction", "indexedLabel").rdd

In [78]:
predictionAndLabels.take(3)

[Row(prediction=0.0, indexedLabel=1.0),
 Row(prediction=0.0, indexedLabel=1.0),
 Row(prediction=0.0, indexedLabel=1.0)]

In [79]:
# Instantiate Basic Metrics object
basicMetrics = MulticlassMetrics(predictionAndLabels)

In [80]:
print("Summary Stats")
print("Weighted Precision = %s" % basicMetrics.weightedPrecision)
print("Weighted Recall = %s" % basicMetrics.weightedRecall)
print("Weighted F1 Score = %s" % basicMetrics.weightedFMeasure())
print("Confusion Matrix:")
pd.DataFrame(basicMetrics.confusionMatrix().toArray())

Summary Stats
Weighted Precision = 0.9845509146020694
Weighted Recall = 0.9922453903153541
Weighted F1 Score = 0.9883831774821917
Confusion Matrix:


,0,1
0,11516.0,0.0
1,90.0,0.0


In [81]:
# Instantiate Advanced Metrics object
advMetrics = BinaryClassificationMetrics(predictionAndLabels)

In [82]:
# Area under precision-recall curve
print("Area under PR = %s" % advMetrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % advMetrics.areaUnderROC)

Area under PR = 0.007754609684645873
Area under ROC = 0.5


In [83]:
# Show rules
treeModel = rfModel.stages[2]
print(treeModel.toDebugString) # summary only

RandomForestClassificationModel (uid=RandomForestClassifier_4a98935fed3b40b06f21) with 10 trees
  Tree 0 (weight 1.0):
    If (feature 7 in {1.0})
     Predict: 0.0
    Else (feature 7 not in {1.0})
     If (feature 42 in {1.0})
      Predict: 0.0
     Else (feature 42 not in {1.0})
      If (feature 84 in {1.0})
       Predict: 0.0
      Else (feature 84 not in {1.0})
       If (feature 0 <= 0.5)
        If (feature 81 in {0.0})
         Predict: 0.0
        Else (feature 81 not in {0.0})
         Predict: 0.0
       Else (feature 0 > 0.5)
        If (feature 30 in {0.0})
         Predict: 0.0
        Else (feature 30 not in {0.0})
         Predict: 0.0
  Tree 1 (weight 1.0):
    If (feature 2 <= 6521.0)
     If (feature 36 in {1.0})
      Predict: 0.0
     Else (feature 36 not in {1.0})
      If (feature 42 in {1.0})
       Predict: 0.0
      Else (feature 42 not in {1.0})
       If (feature 82 in {1.0})
        Predict: 0.0
       Else (feature 82 not in {1.0})
        If (feature 8

### 5.3 Model Selection via Cross-Validation to RandomForest

In [84]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [85]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(connRF)

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 20 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol='features', outputCol="indexedFeatures", maxCategories=20).fit(connRF)
    
# Train a DecisionTree model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

In [86]:
grid = ParamGridBuilder().addGrid(rf.numTrees, range(10, 41, 10)) \
                         .addGrid(rf.maxDepth, range(2, 7, 2)) \
                         .build()

In [87]:
evaluator = BinaryClassificationEvaluator(metricName="areaUnderPR")

In [88]:
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=grid, evaluator=evaluator)

In [89]:
%%time 
cvModel = cv.fit(connRF)

CPU times: user 2.77 s, sys: 1.14 s, total: 3.91 s
Wall time: 1min 24s


In [90]:
%%time
evaluator.evaluate(cvModel.transform(connRF))

CPU times: user 10 ms, sys: 10 ms, total: 20 ms
Wall time: 1.88 s


0.9645267619117346

In [91]:
cvModel.bestModel.stages[2]

RandomForestClassificationModel (uid=RandomForestClassifier_4162ab54719ca87d88ce) with 10 trees

In [92]:
evaluator.evaluate(cvModel.transform(connRF), {evaluator.metricName: "areaUnderPR"})

0.9645267619117346

----------